# Word Embedding

In [1]:
# load keys for Twitter API
key_path = '/content/drive/My Drive/Colab Notebooks/Keys.txt'
f = open(key_path, "r")
consumer_secret, consumer_key = [i.split('\'')[1] for i in f.readlines()]

## Setting up Model

In [7]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [8]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteIMJUMF/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteIMJUMF/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteIMJUMF/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [9]:
encoder = info.features['text'].encoder
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [4025, 222, 6307, 2327, 4043, 2120, 7975]
The original string: "Hello TensorFlow."


In [10]:
for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))

4025 ----> Hell
222 ----> o 
6307 ----> Ten
2327 ----> sor
4043 ----> Fl
2120 ----> ow
7975 ----> .


In [11]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE)

test_dataset = test_dataset.padded_batch(BATCH_SIZE)

In [12]:
model_RNN = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [13]:
model_RNN.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [14]:
history = model_RNN.fit(train_dataset, epochs=10,
                    validation_data=test_dataset, 
                    validation_steps=30)

Epoch 1/10
391/391 [==============================] - 43s 110ms/step - loss: 0.6679 - accuracy: 0.5316 - val_loss: 0.5355 - val_accuracy: 0.7479
Epoch 2/10
391/391 [==============================] - 43s 110ms/step - loss: 0.4455 - accuracy: 0.8012 - val_loss: 0.3825 - val_accuracy: 0.8318
Epoch 3/10
391/391 [==============================] - 43s 110ms/step - loss: 0.2908 - accuracy: 0.8841 - val_loss: 0.3369 - val_accuracy: 0.8609
Epoch 4/10
391/391 [==============================] - 43s 110ms/step - loss: 0.2349 - accuracy: 0.9116 - val_loss: 0.3387 - val_accuracy: 0.8557
Epoch 5/10
391/391 [==============================] - 43s 109ms/step - loss: 0.2039 - accuracy: 0.9262 - val_loss: 0.3541 - val_accuracy: 0.8484
Epoch 6/10
391/391 [==============================] - 43s 109ms/step - loss: 0.1784 - accuracy: 0.9376 - val_loss: 0.3409 - val_accuracy: 0.8703
Epoch 7/10
391/391 [==============================] - 43s 110ms/step - loss: 0.1592 - accuracy: 0.9455 - val_loss: 0.3684 - val_ac

In [15]:
test_loss, test_acc = model_RNN.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

391/391 [==============================] - 18s 47ms/step - loss: 0.4226 - accuracy: 0.8564
Test Loss: 0.42262354493141174
Test Accuracy: 0.856440007686615


In [16]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

def sample_predict(model, sample_pred_text, pad):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)

In [17]:
sample_pred_text = ('Governors Andrew Cuomo of New York has the worst record on death and China Virus. 11,000 people alone died in Nursing Homes because of his incompetence!')
predictions = sample_predict(model_RNN, sample_pred_text, pad=True)
print(predictions)

[[1.4326389]]


Using two LSTMs

In [18]:
model_LSTM = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

model_LSTM.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [19]:
history_LSTM = model_LSTM.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
391/391 [==============================] - 74s 191ms/step - loss: 0.6556 - accuracy: 0.5508 - val_loss: 0.4768 - val_accuracy: 0.7578
Epoch 2/10
391/391 [==============================] - 73s 187ms/step - loss: 0.3472 - accuracy: 0.8578 - val_loss: 0.3385 - val_accuracy: 0.8630
Epoch 3/10
391/391 [==============================] - 73s 186ms/step - loss: 0.2557 - accuracy: 0.9029 - val_loss: 0.3455 - val_accuracy: 0.8630
Epoch 4/10
391/391 [==============================] - 73s 187ms/step - loss: 0.2112 - accuracy: 0.9263 - val_loss: 0.3585 - val_accuracy: 0.8677
Epoch 5/10
391/391 [==============================] - 73s 186ms/step - loss: 0.1816 - accuracy: 0.9398 - val_loss: 0.3917 - val_accuracy: 0.8531
Epoch 6/10
391/391 [==============================] - 74s 189ms/step - loss: 0.1593 - accuracy: 0.9496 - val_loss: 0.4125 - val_accuracy: 0.8557
Epoch 7/10
391/391 [==============================] - 74s 188ms/step - loss: 0.1405 - accuracy: 0.9570 - val_loss: 0.4306 - val_ac

In [20]:
test_loss, test_acc = model_LSTM.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

391/391 [==============================] - 33s 84ms/step - loss: 0.5315 - accuracy: 0.8474
Test Loss: 0.5314837694168091
Test Accuracy: 0.8473600149154663


In [21]:
sample_pred_text = ('Governors Andrew Cuomo of New York has the worst record on death and China Virus. 11,000 people alone died in Nursing Homes because of his incompetence!')
predictions = sample_predict(model_LSTM, sample_pred_text, pad=True)
print(predictions)

[[1.6154921]]


In [23]:
model_RNN.save('my_modelRNN.h5') 
model_LSTM.save('my_modelLSTM.h5')